## Read

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [35]:
reviews =pd.read_table('reviews_sample.txt', header=None, sep='\n')
reviews.shape

(10000, 1)

In [36]:
# setting minimum support
sup = 0.01
min_sup = int(reviews.shape[0]*sup)

In [11]:
# split as list
review_list = reviews[0].map(lambda review: review.split(' '))

## Length 1 pattern

In [194]:
from collections import defaultdict

db = defaultdict(list)

for sid, reviews in enumerate(review_list):
    for eid, review in enumerate(reviews):
        db[review].append([sid,eid])

In [195]:
len(db.keys())

22104

In [196]:
len1db = defaultdict(pd.DataFrame)
for key, items in db.items():
    if len(items) > min_sup:
        len1db[key] = pd.concat([len1db[key], pd.DataFrame(items, columns=['sid','eid' ])], axis=1)

In [197]:
len(len1db.keys())

1092

## Length 2 patterns

In [198]:
def close(data, column1, column2):
    diff = np.absolute(data[column1]-data[column2])
    cond1 = diff<2
    cond2 = diff>0
    return data[cond1 & cond2] 

In [199]:
%%time
# 37min

len2db = defaultdict(pd.DataFrame)
# copy db1
len1dbcopy = len1db.copy()

for key1, items1 in len1db.items():
    for key2, items2 in len1dbcopy.items():
        
        # merge based on key sid
        temp = items1.merge(items2, on='sid', how='inner')
        # see if contigious
        temp = close(temp, 'eid_x', 'eid_y')
        if temp.shape[0]> min_sup:
            len2db[key1+" "+key2] = temp
            
    # remove key from copy db
    len1dbcopy.pop(key1)

Wall time: 37min 19s


In [203]:
len2db.keys()

dict_keys(['like place', 'like really', 'like would', 'like look', 'like looked', 'like feel', 'like felt', 'like tasted', 'year ago', 'selection beer', 'food good', 'food service', 'food great', 'food whole', 'food bar', 'food delicious', 'food quality', 'best one', 'area bar', 'cooked perfectly', 'good service', 'good place', 'good also', 'good really', 'good thing', 'good pretty', 'good price', 'good get', 'good pizza', 'customer service', 'service great', 'service friendly', 'great place', 'great price', 'great experience', 'place nice', 'place pittsburgh', 'place love', 'place get', 'place one', 'staff friendly', 'staff wait', 'make sure', 'special nothing', 'ice cream', 'really really', 'really nice', 'thing one', 'night last', 'night saturday', 'dining room', 'pittsburgh restaurant', 'would definitely', 'would recommend', 'time first', 'time last', 'time every', 'time long', 'time next', 'better much', 'definitely back', 'come back', 'stop dor', 'pretty much', 'lot parking', 'ho

In [213]:
key.split()

['pasta', 'trio']

In [214]:
# divide length 2
db2split = defaultdict()
for key, items in len2db.items():
    
    # items where first key comes first
    idx = items['eid_x']<items['eid_y']
    if items[idx].shape[0] > min_sup:
        db2split[key] = items[idx]
    if items[~idx].shape[0]> min_sup:
        keys = key.split()
        db2split[keys[1]+' '+keys[0]] = items[~idx]

## Length 3 Patterns

In [220]:
%%time
# length 3 db
db3 = defaultdict(pd.DataFrame)

for key1, items1 in db2split.items():
    for key2, items2 in db2split.items():
        # compare key1 first and key2 last
        key1last = key1.split()[1]
        key2first = key2.split()[0]
        key2last = key2.split()[1]
        print(key1last+', '+key2first)
        if key1last==key2first:
            temp = items1.merge(items2, on='sid', how='inner')
            # minimum support check
            if temp.shape[0] >min_sup:
                db3[key1+" "+key2last] = temp

like, place
like, look
like, looked
like, feel
like, felt
like, tasted
like, year
like, beer
like, food
like, good
like, food
like, food
like, great
like, whole
like, one
like, good
like, service
like, good
like, really
like, good
like, pretty
like, customer
like, service
like, great
like, great
like, love
like, place
like, staff
like, friendly
like, wait
like, make
like, nothing
like, ice
like, really
like, last
like, saturday
like, dining
like, would
like, would
like, first
like, last
like, every
like, long
like, next
like, much
like, definitely
like, come
like, dor
like, pretty
like, parking
like, hot
like, fish
like, french
like, next
like, highly
like, reasonable
like, going
like, happy
like, even
like, strip
like, across
like, one
like, coming
like, grocery
like, giant
like, front
like, lobster
like, mac
like, reasonably
like, french
like, cole
like, market
like, pasta
like, place
like, look
like, looked
like, feel
like, felt
like, tasted
like, year
like, beer
like, food
like, go

service, pretty
service, customer
service, service
service, great
service, great
service, love
service, place
service, staff
service, friendly
service, wait
service, make
service, nothing
service, ice
service, really
service, last
service, saturday
service, dining
service, would
service, would
service, first
service, last
service, every
service, long
service, next
service, much
service, definitely
service, come
service, dor
service, pretty
service, parking
service, hot
service, fish
service, french
service, next
service, highly
service, reasonable
service, going
service, happy
service, even
service, strip
service, across
service, one
service, coming
service, grocery
service, giant
service, front
service, lobster
service, mac
service, reasonably
service, french
service, cole
service, market
service, pasta
good, place
good, look
good, looked
good, feel
good, felt
good, tasted
good, year
good, beer
good, food
good, good
good, food
good, food
good, great
good, whole
good, one
good, good
go

nice, cole
nice, market
nice, pasta
night, place
night, look
night, looked
night, feel
night, felt
night, tasted
night, year
night, beer
night, food
night, good
night, food
night, food
night, great
night, whole
night, one
night, good
night, service
night, good
night, really
night, good
night, pretty
night, customer
night, service
night, great
night, great
night, love
night, place
night, staff
night, friendly
night, wait
night, make
night, nothing
night, ice
night, really
night, last
night, saturday
night, dining
night, would
night, would
night, first
night, last
night, every
night, long
night, next
night, much
night, definitely
night, come
night, dor
night, pretty
night, parking
night, hot
night, fish
night, french
night, next
night, highly
night, reasonable
night, going
night, happy
night, even
night, strip
night, across
night, one
night, coming
night, grocery
night, giant
night, front
night, lobster
night, mac
night, reasonably
night, french
night, cole
night, market
night, pasta
nig

stop, pretty
stop, parking
stop, hot
stop, fish
stop, french
stop, next
stop, highly
stop, reasonable
stop, going
stop, happy
stop, even
stop, strip
stop, across
stop, one
stop, coming
stop, grocery
stop, giant
stop, front
stop, lobster
stop, mac
stop, reasonably
stop, french
stop, cole
stop, market
stop, pasta
much, place
much, look
much, looked
much, feel
much, felt
much, tasted
much, year
much, beer
much, food
much, good
much, food
much, food
much, great
much, whole
much, one
much, good
much, service
much, good
much, really
much, good
much, pretty
much, customer
much, service
much, great
much, great
much, love
much, place
much, staff
much, friendly
much, wait
much, make
much, nothing
much, ice
much, really
much, last
much, saturday
much, dining
much, would
much, would
much, first
much, last
much, every
much, long
much, next
much, much
much, definitely
much, come
much, dor
much, pretty
much, parking
much, hot
much, fish
much, french
much, next
much, highly
much, reasonable
much, goin

favorite, love
favorite, place
favorite, staff
favorite, friendly
favorite, wait
favorite, make
favorite, nothing
favorite, ice
favorite, really
favorite, last
favorite, saturday
favorite, dining
favorite, would
favorite, would
favorite, first
favorite, last
favorite, every
favorite, long
favorite, next
favorite, much
favorite, definitely
favorite, come
favorite, dor
favorite, pretty
favorite, parking
favorite, hot
favorite, fish
favorite, french
favorite, next
favorite, highly
favorite, reasonable
favorite, going
favorite, happy
favorite, even
favorite, strip
favorite, across
favorite, one
favorite, coming
favorite, grocery
favorite, giant
favorite, front
favorite, lobster
favorite, mac
favorite, reasonably
favorite, french
favorite, cole
favorite, market
favorite, pasta
back, place
back, look
back, looked
back, feel
back, felt
back, tasted
back, year
back, beer
back, food
back, good
back, food
back, food
back, great
back, whole
back, one
back, good
back, service
back, good
back, real

## Save

In [230]:
# combine all dbs
results = len1db.copy()
results.update(len2db)
results.update(db3)

In [233]:
with open("patterns.txt", 'w') as file:
    # length 1
    for key, items in results.items():
        support = items.shape[0]
        file.write(str(support)+":"+';'.join(key.split())+ '\n')
